# Landing Performance - Exercise 2

Of a BAE 146 series 200, the following data are known:
- Aircraft mass, $ m = 36741 \text{ kg} $
- Wing area, $ S = 77.3 \text{ } m^2 $
- Drag coefficients, $ C_D = 0.08 + 0.044 \cdot C_L^2 $
- Max lift coefficient, $ C_{L_{max}} = 2.52
- Approach glide path angle, $ \gamma = 3 \degree $
- Appraoch speed, $ v_{ref} = 1.3 \cdot v_{min} $
- Touchdown speed, $ v_{td} = 1.15 \cdot v_{min} $
- Thrust at landing, $ T = 0 \text{ N}$
- Air density = $ \rho = 1.225 \text{ } \frac{kg}{m^3} $
- During braking on ground run:
    - lift coefficient, $ C_L = 1.2 $
    - friction coefficient, $ \mu = 0.4 $

In [2]:
import math
import qty

### Initialize variables

In [15]:
mass = qty.Mass(36741)
surface = qty.Area(77.3)
cd_0 = 0.08
cd_i_k = 0.044
cl_max = 2.52
fpa = math.radians(3)
v_min_to_vref = 1.3
v_min_to_vtd = 1.15
thrust_td = 0
density = 1.225

screen_height = qty.Distance.Ft(50)
flare_height = qty.Distance.Ft(30)

cl_gnd = 1.2
friction = 0.4

### Minimum speed

$ \large v_{min} = \sqrt{\frac{2 \cdot W}{\rho \cdot S \cdot C_{L_{max}}}} $

In [5]:
weight = mass * 9.81

v_min = qty.Velocity(math.sqrt((2 * weight) / (density * surface * cl_max)))
v_ref = qty.Velocity(v_min_to_vref * v_min)
v_td = qty.Velocity(v_min_to_vtd * v_min)

print(v_min, v_ref, v_td)

54.96 m/s 71.45 m/s 63.21 m/s


### Approach Distance

#### Stationary part

$ \large S_A = \frac{h_A - h_f}{\tan{{\gamma}_A}} $

In [8]:
stationary_distance = qty.Distance((screen_height - flare_height) / (math.tan(fpa)))

print(stationary_distance)

116.32 meters


#### Flare part

$ \large S_B = \frac{\frac{W}{2g}({TAS}_C^2 - {TAS}_B^2) - W \cdot h_f}{T - D} $

where (T -D) is the average between point B and C. Point B is at the start of the flare:

$ D - T = W \cdot \sin{\gamma} $


In [10]:
t_min_d_b = qty.Force(-(weight * math.sin(fpa)))

print(t_min_d_b)

-18863.41 Newtons


At touchdown:

In [19]:
cl_td = (2 * weight) / (density * v_td**2 * surface)

cd_td = cd_0 + cd_i_k * cl_td**2

drag_td = qty.Force(0.5 * density * v_td**2 * cd_td * surface)

t_min_d_c = qty.Force(thrust_td - drag_td)

t_min_d_avg = qty.Force((t_min_d_b + t_min_d_c) / 2)

print(cl_td, cd_td, drag_td, t_min_d_c, t_min_d_avg)

1.9054820415879024 0.239757919675816 45351.13 Newtons -45351.13 Newtons -32107.27 Newtons


With these values, the flare distance can be obtained

In [21]:
flare_dist = qty.Distance((weight / (2 * 9.81) * (v_td**2 - v_ref**2) - (weight * flare_height)) / t_min_d_avg)

print(flare_dist)

737.85 meters
